# Tutorial 2: Double Pendulum and Chaos

Explore chaotic dynamics with the double pendulum system.

In [ ]:
from mechanics_dsl import PhysicsCompiler
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dsl_code = r"""
\system{double_pendulum}

\defvar{theta1}{Angle}{rad}
\defvar{theta2}{Angle}{rad}
\defvar{m1}{Mass}{kg}
\defvar{m2}{Mass}{kg}
\defvar{l1}{Length}{m}
\defvar{l2}{Length}{m}
\defvar{g}{Acceleration}{m/s^2}

\parameter{m1}{1.0}{kg}
\parameter{m2}{1.0}{kg}
\parameter{l1}{1.0}{m}
\parameter{l2}{1.0}{m}
\parameter{g}{9.81}{m/s^2}

\lagrangian{
    \frac{1}{2} * (m1 + m2) * l1^2 * \dot{theta1}^2
    + \frac{1}{2} * m2 * l2^2 * \dot{theta2}^2
    + m2 * l1 * l2 * \dot{theta1} * \dot{theta2} * \cos{theta1 - theta2}
    - (m1 + m2) * g * l1 * (1 - \cos{theta1})
    - m2 * g * l2 * (1 - \cos{theta2})
}

\initial{theta1=2.5, theta1_dot=0.0, theta2=2.0, theta2_dot=0.0}
"""

compiler = PhysicsCompiler()
result = compiler.compile_dsl(dsl_code)
print(f"Compiled: {result['system_name']}")

In [ ]:
# Simulate
solution = compiler.simulate(t_span=(0, 20), num_points=2000)

# Plot angles
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(solution['t'], solution['y'][0], 'b-', lw=1)
axes[0, 0].set_ylabel('theta1 (rad)')
axes[0, 0].set_title('First Pendulum Angle')

axes[0, 1].plot(solution['t'], solution['y'][2], 'r-', lw=1)
axes[0, 1].set_ylabel('theta2 (rad)')
axes[0, 1].set_title('Second Pendulum Angle')

axes[1, 0].plot(solution['y'][0], solution['y'][1], 'b-', lw=0.5, alpha=0.7)
axes[1, 0].set_xlabel('theta1')
axes[1, 0].set_ylabel('theta1_dot')
axes[1, 0].set_title('Phase Space (Pendulum 1)')

axes[1, 1].plot(solution['y'][2], solution['y'][3], 'r-', lw=0.5, alpha=0.7)
axes[1, 1].set_xlabel('theta2')
axes[1, 1].set_ylabel('theta2_dot')
axes[1, 1].set_title('Phase Space (Pendulum 2)')

for ax in axes.flat:
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Sensitivity to Initial Conditions

Chaotic systems are sensitive to initial conditions. A tiny change leads to completely different behavior.

In [ ]:
# Run with slightly different initial conditions
compiler.simulator.initial_conditions['theta1'] = 2.5
sol1 = compiler.simulate(t_span=(0, 20), num_points=2000)

compiler.simulator.initial_conditions['theta1'] = 2.501  # 0.1% difference
sol2 = compiler.simulate(t_span=(0, 20), num_points=2000)

# Compare
diff = np.abs(sol1['y'][0] - sol2['y'][0])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))

ax1.plot(sol1['t'], sol1['y'][0], 'b-', label='theta1(0) = 2.500', alpha=0.7)
ax1.plot(sol2['t'], sol2['y'][0], 'r-', label='theta1(0) = 2.501', alpha=0.7)
ax1.legend()
ax1.set_ylabel('theta1 (rad)')
ax1.set_title('Sensitivity to Initial Conditions')

ax2.semilogy(sol1['t'], diff + 1e-10, 'k-')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('|Difference|')
ax2.set_title('Divergence Over Time')

for ax in [ax1, ax2]:
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Cartesian Trajectory

Convert angles to x-y coordinates to visualize the pendulum path.

In [ ]:
l1, l2 = 1.0, 1.0
theta1 = sol1['y'][0]
theta2 = sol1['y'][2]

# Position of first mass
x1 = l1 * np.sin(theta1)
y1 = -l1 * np.cos(theta1)

# Position of second mass
x2 = x1 + l2 * np.sin(theta2)
y2 = y1 - l2 * np.cos(theta2)

plt.figure(figsize=(8, 8))
plt.plot(x2, y2, 'b-', lw=0.5, alpha=0.5)
plt.scatter([x2[0]], [y2[0]], c='green', s=100, zorder=5, label='Start')
plt.scatter([x2[-1]], [y2[-1]], c='red', s=100, zorder=5, label='End')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Trajectory of Second Pendulum Mass')
plt.axis('equal')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()